# Linear models with *CNN* features

In [1]:
%matplotlib inline

import numpy as np
import os,json
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from utils import plots,get_batches,plot_confusion_matrix,get_data
import keras
from keras.utils.data_utils import get_file
from keras.layers.core import Dropout,Flatten,Dense,Lambda
from keras.layers.convolutional import ZeroPadding2D,MaxPooling2D,Conv2D
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Input
from keras.preprocessing import image

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Example Working

In [ ]:
lm=Sequential([Dense(1,input_shape=(2,))])

In [ ]:
lm.summary()

In [ ]:
x=np.random.random((30,2))

In [ ]:
y=np.dot(x,[2,3])+1

In [ ]:
lm.compile(optimizer=keras.optimizers.SGD(lr=0.1),loss='mse')

In [ ]:
lm.evaluate(x,y)

In [ ]:
lm.fit(x,y,epochs=5,batch_size=1)

In [ ]:
lm.evaluate(x,y)

In [ ]:
lm.get_weights()

## Training Linear predictions

In [2]:
path='../dogscats/sample/'
model_path = path+'models/'

if not os.path.exists(model_path) : os.mkdir(model_path)

In [3]:
batch_size=4

In [4]:
from vgg16 import Vgg16
vgg = Vgg16()
model = vgg.model

In [5]:
batches=get_batches(path+'train',shuffle=False,batch_size=4)
val_batches=get_batches(path+'valid',shuffle=False,batch_size=4)

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [ ]:
batches.

In [6]:
trn_data=get_data(path+'train')

Found 16 images belonging to 2 classes.


In [7]:
from utils import get_data

In [7]:
trn_data=get_data(path+'train')
val_data=get_data(path+'valid')

Found 16 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [12]:
trn_data[0].min()

0.0

In [11]:
def onehot(x): return np.array(OneHotEncoder().fit_transform(x.reshape(-1,1)).todense())

In [12]:
val_classes=val_batches.classes
trn_classes=batches.classes

val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

In [13]:
trn_labels.shape

(16, 2)

In [16]:
trn_classes[:4]

array([0, 0, 0, 0], dtype=int32)

In [17]:
trn_labels[:4]

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [18]:
trn_features=model.predict(trn_data,batch_size=batch_size)
val_features=model.predict(val_data,batch_size=batch_size)

In [19]:
trn_features.shape

(16, 1000)

In [20]:
lm=Sequential([Dense(2,activation='softmax',input_shape=(1000,))])

In [21]:
lm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 2)                 2002      
Total params: 2,002
Trainable params: 2,002
Non-trainable params: 0
_________________________________________________________________


In [22]:
lm.compile(optimizer=Adam(),loss='categorical_crossentropy',metrics=['accuracy'])

In [23]:
batch_size

4

In [25]:
lm.fit(trn_features,trn_labels,epochs=4,batch_size=batch_size,
       validation_data=(val_features,val_labels))

Train on 16 samples, validate on 8 samples
Epoch 1/4
16/16 [==============================] - 0s - loss: 0.6950 - acc: 0.4375 - val_loss: 0.6922 - val_acc: 0.5000
Epoch 2/4
16/16 [==============================] - 0s - loss: 0.6933 - acc: 0.5625 - val_loss: 0.6908 - val_acc: 0.5000
Epoch 3/4
16/16 [==============================] - 0s - loss: 0.6918 - acc: 0.6250 - val_loss: 0.6893 - val_acc: 0.6250
Epoch 4/4
16/16 [==============================] - 0s - loss: 0.6902 - acc: 0.7500 - val_loss: 0.6879 - val_acc: 0.6250


In [30]:
lm.predict_classes(val_features)

8/8 [==============================] - 0s


array([1, 0, 0, 0, 1, 1, 0, 0])

In [35]:
np.shape(val_data[0])

(3, 224, 224)

In [43]:
vgg.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [44]:
model.pop()

In [45]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [49]:
layers=model.layers

In [59]:
first_Dense_idx = [index for index,layer in enumerate(layers) if type(layer) is Dense][0]

In [62]:
for layer in layers[first_Dense_idx:]:layer.trainable=True

In [63]:
from keras import backend as K

In [65]:
opt=keras.optimizers.RMSprop(lr=0.1)
K.set_value(opt.lr,0.01)

In [70]:
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [71]:
model.fit_generator(batches,steps_per_epoch=batches.n,epochs=3,validation_data=val_batches,validation_steps=val_batches.n)

Epoch 1/3


ValueError: Error when checking target: expected dropout_2 to have shape (None, 4096) but got array with shape (4, 2)

In [72]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________